In [14]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%autoreload 2

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation, Neptune_data_read
from parameter import *
from dspike_formulas import *



In [17]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-02-17/"
files_start = 3280
files_end = 3289

In [18]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["116", "117", "118", "119", "120", "122", "124"]
cup_config = cycle_Sb_2

# Mass Range of cup configuration
mass_range = cycle_Sb_2_mass_range

# Isotopes used for Interference correction
#corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Te" : "125", "Xe" : "129"}
#corr_isotopes_2 = {}
corr_isotopes_Sb = {"Te": "125"}

norm_ratio_int = ["122","118"]
norm_ratio_Sb = ["123", "121"]
denom_isotope_int = norm_ratio_int[1]

Sn_monitor = ["125", "129"]


def eval_iso_list(isotopes_list,norm_ratio, monitor_iso):
    isotope_nom = norm_ratio[0]
    isotope_den = norm_ratio[1]
    isotopes_list.remove(norm_ratio[1])
    for i in range(len(monitor_iso)):
        isotopes_list.append(monitor_iso[i])
    isotopes_list.sort()
    return isotopes_list

isotope_ls = eval_iso_list(Sn_isotopes, norm_ratio_int, Sn_monitor)

data_sample_column = [(i + "/" + denom_isotope_int) for i in isotope_ls]    
isotopes = [isotope_ls]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

In [76]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(6411,6518,1) + range(6526,6534, 1) + range(6537,6577, 1)

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2016-02-18 15:20:00,1,NIST 100ppb,0.015554,0.015718,-2.703260e-05,2.194838,4.999298,1.714318,6.522626,11.777094,3.064079,8.533992,2.668879,4.995175,0.114544
2016-02-18 15:20:00,2,NIST 100ppb,0.015569,0.015781,-6.402400e-06,2.191916,4.993021,1.712166,6.514293,11.762257,3.060482,8.523024,2.665691,4.988804,0.114367
2016-02-18 15:20:00,3,NIST 100ppb,0.015631,0.015811,-1.067080e-05,2.209219,5.031175,1.725553,6.564404,11.854426,3.084206,8.589822,2.686454,5.027743,0.115234
2016-02-18 15:20:00,4,NIST 100ppb,0.015459,0.015619,-8.536500e-06,2.193890,4.996323,1.713591,6.518955,11.772395,3.063221,8.531298,2.668301,4.993442,0.114499
2016-02-18 15:20:00,5,NIST 100ppb,0.015545,0.015666,-1.493910e-05,2.205805,5.023537,1.722973,6.554481,11.835799,3.079838,8.577171,2.682525,5.020513,0.115105
2016-02-18 15:20:00,6,NIST 100ppb,0.015522,0.015679,-9.959300e-06,2.220737,5.057928,1.734688,6.599043,11.917279,3.100684,8.635455,2.700780,5.054756,0.115936
2016-02-18 15:20:00,7,NIST 100ppb,0.015493,0.015603,-5.691100e-06,2.223379,5.066045,1.736715,6.610043,11.931285,3.104168,8.645824,2.703956,5.060962,0.116064
2016-02-18 15:20:00,8,NIST 100ppb,0.015645,0.015749,-3.058940e-05,2.242902,5.107904,1.751846,6.664188,12.033996,3.131482,8.721231,2.727533,5.103961,0.117046
2016-02-18 15:20:00,9,NIST 100ppb,0.015602,0.015744,1.280490e-05,2.243671,5.110140,1.752268,6.668172,12.039496,3.132168,8.723572,2.728101,5.106193,0.117100


In [77]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()
df_Sb_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
    df_zero = df.data_zero_corr(sample)
    new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    new_corr.line2_corr(df_zero, "119")
    
    data_int = new_corr.internal_norm_1(norm_ratio_int, denom_isotope_int, iter_beta)
    beta_int = new_corr.beta("Sn", norm_ratio_int, iter_beta)
    data_int = pd.DataFrame.from_dict(data_int, orient = 'index')
    beta_int = pd.Series(beta_int, name="beta_int")
    data_int = new_corr.mad_outlier_rejection(data_int)
    
    data_Sb = new_corr.external_norm_Sb(norm_ratio_Sb, denom_isotope_int, iter_beta)
    beta_Sb = new_corr.beta("Sb", norm_ratio_Sb, iter_beta)
    data_Sb = pd.DataFrame.from_dict(data_Sb, orient = 'index')
    beta_Sb= pd.Series(beta_Sb, name="beta_Sb")
    data_Sb = new_corr.mad_outlier_rejection(data_Sb)
   
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    
    #data_int = data_int.drop(denom_isotope_int, axis=1)
    data_int = data_int[sorted(data_int.columns)]
    data_int.columns = data_sample_column
    data_int["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_int["sample"] = sample_name
    data_int["cycle"] = data_int.index.values
    data_int = pd.concat((data_int, beta_int), axis=1, join='inner')
    
    #data_Sb = data_Sb.drop(denom_isotope_int, axis=1)
    data_Sb = data_Sb[sorted(data_Sb.columns)]
    data_Sb.columns = data_sample_column
    data_Sb["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_Sb["sample"] = sample_name
    data_Sb["cycle"] = data_Sb.index.values
    data_Sb= pd.concat((data_Sb, beta_Sb), axis=1, join='inner')

    df_internal_norm = df_internal_norm.append(data_int)
    df_Sb_norm = df_Sb_norm.append(data_Sb)
df_internal_norm.set_index("date", inplace = True)
df_Sb_norm.set_index("date", inplace = True)

In [116]:
True_123Sb_121Sb = 0.7479
mod_123Sb_121Sb = 0.74814

In [117]:
df_Sb_norm["124/118"].mean() - df_internal_norm["124/118"].mean()

-0.00022920019155900362

In [118]:
df_Sb_norm_new = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    
    df_zero = df.data_zero_corr(sample)
    new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    new_corr.line2_corr(df_zero, "119")
        
    data_Sb_new = new_corr.external_norm_Sb_change_true(norm_ratio_Sb, denom_isotope_int, iter_beta, mod_123Sb_121Sb)
    data_Sb_new = pd.DataFrame.from_dict(data_Sb_new, orient = 'index')
    data_Sb_new = new_corr.mad_outlier_rejection(data_Sb_new)
        
    data_Sb_new = data_Sb_new[sorted(data_Sb_new.columns)]
    data_Sb_new.columns = data_sample_column
    data_Sb_new["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_Sb_new["sample"] = sample_name
    data_Sb_new["cycle"] = data_Sb_new.index.values
        
    df_Sb_norm_new = df_Sb_norm_new.append(data_Sb_new)
    
df_Sb_norm_new.set_index("date", inplace = True)


In [119]:
df_Sb_norm_new

,116/118,117/118,119/118,120/118,122/118,124/118,125/118,129/118,sample,cycle
date,,,,,,,,,,
2016-02-18 15:20:00,0.600273,0.316681,0.354595,1.346219,0.191239,0.239030,-2.909396e-06,0.001598,NIST 100ppb,1
2016-02-18 15:20:00,0.600293,0.316713,0.354630,1.346223,0.191235,0.239001,-6.898959e-07,0.001601,NIST 100ppb,2
2016-02-18 15:20:00,0.600242,0.316689,0.354610,1.346291,0.191252,0.239052,-1.141102e-06,0.001596,NIST 100ppb,3
2016-02-18 15:20:00,0.600234,0.316707,0.354613,1.346152,0.191230,0.239024,-9.191407e-07,0.001589,NIST 100ppb,4
2016-02-18 15:20:00,0.600259,0.316692,0.354631,1.346167,0.191250,0.239040,-1.599934e-06,0.001589,NIST 100ppb,5
2016-02-18 15:20:00,0.600303,0.316702,0.354613,1.346222,0.191232,0.238999,-1.059238e-06,0.001576,NIST 100ppb,6
2016-02-18 15:20:00,0.600281,0.316684,0.354597,1.346273,0.191250,0.239041,-6.046926e-07,0.001572,NIST 100ppb,7
2016-02-18 15:20:00,0.600191,0.316695,0.354611,1.346024,0.191222,0.239005,-3.221284e-06,0.001573,NIST 100ppb,8
2016-02-18 15:20:00,0.600191,NaN,0.354623,1.346500,0.191277,NaN,1.348850e-06,0.001569,NIST 100ppb,9


In [120]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_Sb_norm_sub = df_Sb_norm.drop("cycle", axis=1).groupby(df_Sb_norm.index).agg([np.mean, SD2])
df_Sb_norm_sub["sample"] = df_Sb_norm[~df_Sb_norm.index.duplicated(keep='first')]["sample"]
df_Sb_norm_new_sub = df_Sb_norm_new.drop("cycle", axis=1).groupby(df_Sb_norm_new.index).agg([np.mean, SD2])
df_Sb_norm_new_sub["sample"] = df_Sb_norm_new[~df_Sb_norm_new.index.duplicated(keep='first')]["sample"]
df_internal_norm_sub = df_internal_norm.drop("cycle", axis=1).groupby(df_internal_norm.index).agg([np.mean, SD2])
df_internal_norm_sub["sample"] = df_internal_norm[~df_internal_norm.index.duplicated(keep='first')]["sample"]

In [121]:
df_Sb_norm_sub

116/118             117/118             119/118  \
                         mean       SD2      mean       SD2      mean   
date                                                                    
2016-02-18 15:20:00  0.600459  0.000062  0.316757  0.000027  0.354561   
2016-02-18 15:36:00  0.600460  0.000065  0.316762  0.000028  0.354567   
2016-02-18 15:44:00  0.600454  0.000067  0.316763  0.000030  0.354577   
2016-02-18 15:53:00  0.600460  0.000073  0.316766  0.000034  0.354575   
2016-02-18 16:02:00  0.600456  0.000061  0.316763  0.000037  0.354576   
2016-02-18 16:11:00  0.600456  0.000058  0.316767  0.000037  0.354575   
2016-02-18 16:26:00  0.600455  0.000064  0.316765  0.000039  0.354575   

                                120/118             122/118            \
                          SD2      mean       SD2      mean       SD2   
date                                                                    
2016-02-18 15:20:00  0.000023  1.345753  0.000203  0.191120  0.000034   
2016-02-18 15:36:00  0.000031  1.345741  0.000221  0.191122  0.000030   
2016-02-18 15:44:00  0.000034  1.345741  0.000250  0.191129  0.000033   
2016-02-18 15:53:00  0.000030  1.345746  0.000253  0.191129  0.000035   
2016-02-18 16:02:00  0.000026  1.345718  0.000262  0.191127  0.000032   
2016-02-18 16:11:00  0.000034  1.345736  0.000220  0.191129  0.000030   
2016-02-18 16:26:00  0.000034  1.345723  0.000193  0.191129  0.000029   

                      124/118                 125/118             129/118  \
                         mean       SD2          mean       SD2      mean   
date                                                                        
2016-02-18 15:20:00  0.238791  0.000055 -1.306817e-06  0.000003  0.001565   
2016-02-18 15:36:00  0.238789  0.000065 -7.901868e-07  0.000003  0.001536   
2016-02-18 15:44:00  0.238792  0.000066 -9.104211e-07  0.000003  0.001574   
2016-02-18 15:53:00  0.238788  0.000062 -1.054216e-06  0.000003  0.001514   
2016-02-18 16:02:00  0.238782  0.000059 -9.100246e-07  0.000003  0.001531   
2016-02-18 16:11:00  0.238781  0.000057 -6.900937e-07  0.000003  0.001577   
2016-02-18 16:26:00  0.238781  0.000064 -1.094716e-07  0.000004  0.001590   

                                beta_Sb                 sample  
                          SD2      mean       SD2               
date                                                            
2016-02-18 15:20:00  0.000041 -1.492157  0.004994  NIST 100ppb  
2016-02-18 15:36:00  0.000032 -1.498323  0.005793  NIST 100ppb  
2016-02-18 15:44:00  0.000041 -1.497910  0.007107  NIST 100ppb  
2016-02-18 15:53:00  0.000040 -1.497168  0.004992  NIST 100ppb  
2016-02-18 16:02:00  0.000046 -1.496906  0.004378  NIST 100ppb  
2016-02-18 16:11:00  0.000040 -1.497147  0.005797  NIST 100ppb  
2016-02-18 16:26:00  0.000046 -1.492234  0.004659  NIST 100ppb

In [122]:
df_Sb_norm_new_sub

116/118             117/118             119/118  \
                         mean       SD2      mean       SD2      mean   
date                                                                    
2016-02-18 15:20:00  0.600258  0.000062  0.316705  0.000027  0.354620   
2016-02-18 15:36:00  0.600259  0.000065  0.316710  0.000028  0.354626   
2016-02-18 15:44:00  0.600253  0.000067  0.316710  0.000030  0.354636   
2016-02-18 15:53:00  0.600260  0.000073  0.316714  0.000034  0.354634   
2016-02-18 16:02:00  0.600255  0.000061  0.316710  0.000037  0.354635   
2016-02-18 16:11:00  0.600255  0.000058  0.316715  0.000037  0.354634   
2016-02-18 16:26:00  0.600254  0.000064  0.316713  0.000039  0.354633   

                                120/118             122/118            \
                          SD2      mean       SD2      mean       SD2   
date                                                                    
2016-02-18 15:20:00  0.000023  1.346196  0.000203  0.191245  0.000034   
2016-02-18 15:36:00  0.000031  1.346183  0.000221  0.191247  0.000030   
2016-02-18 15:44:00  0.000034  1.346184  0.000250  0.191254  0.000033   
2016-02-18 15:53:00  0.000030  1.346189  0.000253  0.191254  0.000035   
2016-02-18 16:02:00  0.000026  1.346161  0.000262  0.191252  0.000032   
2016-02-18 16:11:00  0.000034  1.346179  0.000220  0.191254  0.000030   
2016-02-18 16:26:00  0.000034  1.346166  0.000193  0.191254  0.000029   

                      124/118                 125/118             129/118  \
                         mean       SD2          mean       SD2      mean   
date                                                                        
2016-02-18 15:20:00  0.239023  0.000055 -1.308292e-06  0.000003  0.001568   
2016-02-18 15:36:00  0.239021  0.000065 -7.910787e-07  0.000003  0.001538   
2016-02-18 15:44:00  0.239024  0.000066 -9.114488e-07  0.000003  0.001577   
2016-02-18 15:53:00  0.239020  0.000062 -1.055406e-06  0.000003  0.001517   
2016-02-18 16:02:00  0.239014  0.000059 -9.110518e-07  0.000003  0.001534   
2016-02-18 16:11:00  0.239013  0.000057 -6.908727e-07  0.000003  0.001580   
2016-02-18 16:26:00  0.239013  0.000064 -1.095952e-07  0.000004  0.001592   

                                    sample  
                          SD2               
date                                        
2016-02-18 15:20:00  0.000041  NIST 100ppb  
2016-02-18 15:36:00  0.000033  NIST 100ppb  
2016-02-18 15:44:00  0.000042  NIST 100ppb  
2016-02-18 15:53:00  0.000040  NIST 100ppb  
2016-02-18 16:02:00  0.000046  NIST 100ppb  
2016-02-18 16:11:00  0.000040  NIST 100ppb  
2016-02-18 16:26:00  0.000046  NIST 100ppb

In [123]:
df_internal_norm_sub

116/118             117/118             119/118  \
                         mean       SD2      mean       SD2      mean   
date                                                                    
2016-02-18 15:20:00  0.600253  0.000060  0.316702  0.000026  0.354623   
2016-02-18 15:36:00  0.600255  0.000074  0.316709  0.000027  0.354627   
2016-02-18 15:44:00  0.600264  0.000078  0.316712  0.000029  0.354634   
2016-02-18 15:53:00  0.600266  0.000073  0.316716  0.000032  0.354631   
2016-02-18 16:02:00  0.600259  0.000079  0.316710  0.000030  0.354634   
2016-02-18 16:11:00  0.600262  0.000067  0.316716  0.000031  0.354631   
2016-02-18 16:26:00  0.600261  0.000063  0.316715  0.000039  0.354631   

                                120/118            122/118                \
                          SD2      mean       SD2     mean           SD2   
date                                                                       
2016-02-18 15:20:00  0.000027  1.346213  0.000161  0.19125  0.000000e+00   
2016-02-18 15:36:00  0.000026  1.346192  0.000176  0.19125  5.551115e-17   
2016-02-18 15:44:00  0.000028  1.346168  0.000193  0.19125  0.000000e+00   
2016-02-18 15:53:00  0.000024  1.346174  0.000205  0.19125  5.551115e-17   
2016-02-18 16:02:00  0.000019  1.346152  0.000198  0.19125  0.000000e+00   
2016-02-18 16:11:00  0.000028  1.346165  0.000169  0.19125  5.551115e-17   
2016-02-18 16:26:00  0.000030  1.346152  0.000178  0.19125  5.551115e-17   

                      124/118                 125/118             129/118  \
                         mean       SD2          mean       SD2      mean   
date                                                                        
2016-02-18 15:20:00  0.239035  0.000040 -1.308415e-06  0.000003  0.001568   
2016-02-18 15:36:00  0.239025  0.000040 -7.910786e-07  0.000003  0.001538   
2016-02-18 15:44:00  0.239016  0.000041 -9.113898e-07  0.000003  0.001577   
2016-02-18 15:53:00  0.239012  0.000037 -1.055357e-06  0.000003  0.001517   
2016-02-18 16:02:00  0.239010  0.000036 -9.111011e-07  0.000003  0.001534   
2016-02-18 16:11:00  0.239006  0.000036 -6.908694e-07  0.000003  0.001580   
2016-02-18 16:26:00  0.239005  0.000038 -1.095666e-07  0.000004  0.001592   

                               beta_int                 sample  
                          SD2      mean       SD2               
date                                                            
2016-02-18 15:20:00  0.000042 -1.471834  0.003177  NIST 100ppb  
2016-02-18 15:36:00  0.000032 -1.478390  0.003464  NIST 100ppb  
2016-02-18 15:44:00  0.000041 -1.479058  0.005978  NIST 100ppb  
2016-02-18 15:53:00  0.000040 -1.478271  0.003901  NIST 100ppb  
2016-02-18 16:02:00  0.000046 -1.477746  0.003455  NIST 100ppb  
2016-02-18 16:11:00  0.000040 -1.478212  0.003240  NIST 100ppb  
2016-02-18 16:26:00  0.000046 -1.473364  0.002875  NIST 100ppb